# Heuristic Approach 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [4]:
data = pd.read_csv("03-pages_cleaned.csv")
# Ensure text columns are strings
data['page1'] = data['page1'].astype(str)
data['page2'] = data['page2'].astype(str)

## Define Heuristic Logic
We define Regex patterns to capture:
1. **Abstract**
2. **Keywords**

In [5]:
from utils.preprocessing import extract_abstract ,extract_keywords

### Apply Extraction

In [6]:

# 1. Extract Abstract
data['abstract_ar'] = data.progress_apply(extract_abstract, axis=1)

# 2. Extract Keywords
data['keywords'] = data.progress_apply(extract_keywords, axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1624/1624 [00:00<00:00, 1875.33it/s]


In [7]:
data.head()

,article_id,title,abstract_ar,authors,authors_en,general_field,publish_date,page1,page2,source,path,keywords
0,1,درجة اكتساب طلبة كلية التربية بجامعة السلطان ق...,هدفت هذه الدراسه الا الكشف عن درحه اكتساب طلبه...,"[""نور النجار""]",NaN,NaN,NaN,Psychology\nVolume 23 Issue 2 Article 1\nلمهار...,مجله اتحاد الجامعات العربيه للتربيه وعلم النفس...,AJP,finalpdfs/AJP_1337_1.pdf,[كليه التربيه البحث الاجرائي جامعه السلطان قاب...
1,2,القيادة الاستباقية وعلاقتها بإدارة مخاطر البيئ...,هدفت هذه الدراسه التعرف الا days ممارسه القياد...,"[""لانا عبد المولى العنزي"", ""محمد عبود الحراحشة""]",NaN,NaN,NaN,Psychology\nVolume 23 Issue 2 Article 2\nSU ال...,القياده الاستباقيه وعلاقتها باداره مخاطر البيئ...,AJP,finalpdfs/AJP_1338_2.pdf,[القياده الاستباقيه اداره bE البيئه المدرسيه م...
2,3,درجة ممارسة معلمي اللغة العربية لمهارات التذوق...,هدفت الدراسه التعرف الا درحه ممارسه معلمي اللغ...,"[""أمل مد الله الشعراء""]",NaN,NaN,NaN,Psychology\nVolume 23 Issue 2 Article 3\na 9 م...,مجله اتحاد الجامعات العربيه للتربيه وعلم النفس...,AJP,finalpdfs/AJP_1339_3.pdf,[مهارات التذوق SLA النصوص الشعريه معلمي اللغه ...
3,4,تحليل محتوى منهاج الرياضيات للصف الثامن من مرح...,هدف البحث الا تحليل محتوا منهاج الرياضيات للصف...,"[""ريتا سعيد""]",NaN,NaN,NaN,Psychology\nVolume 23 Issue 2 Article 4\nritas...,تحليل محتوا منهاج الرياضيات للصف الثامن من مرح...,AJP,finalpdfs/AJP_1340_4.pdf,[تحليل cst محتوا منهاج الرياضيات ples التمثيل ...
4,5,تصميم تعليمي وفق التعلم الخبراتي وأثره في التح...,هدف البحث الا تعرف اثر تصميم تعليمي وفق التعلم...,"[""سلوى محسن العامري""]",NaN,NaN,NaN,Psychology\nVolume 23 Issue 2 Article 5\nتصميم...,مجله اتحاد الجامعات العربيه للتربيه وعلم النفس...,AJP,finalpdfs/AJP_1341_5.pdf,[تصميم تعليميء التعلم الخبراق التحصيلء كفاءه ا...


In [8]:
data.head(1)["page1"].iloc[0]

'Psychology\nVolume 23 Issue 2 Article 1\nلمهارات البحث الاجرائي وصعوبات تنفيذه من خلال اعدادهم\nThe Level of Acquisition of Action Research .لمشروع التخرجح\nSkills and Difficulties in its preperarions Through Graduation \nProject Among College of Education Students at Sultan Qaboos \nUniversity \nنور النجار\nnour.najar@seciauni.org ,كليه التربيه,. جامعه السلطان pools عمان\nDigibalrt of the Education Commons\nCommons\nNetwork ا \nRecommended Citation\nالنجار, نور )2025( درجه اكتساب طلبه كليه av yall بجامعه السلطان قابوس لمهارات البحث الاجرائي وص 992ب\nThe Level of Acquisition of Action Research Skills and Difficulties in .تنفيذه من خلال اعدادهم لمشروع التخرج\nits preperarions Through Graduation Project Among College of Education Students at Sultan Qaboos \nUniversity, Association of Arab Universities Journal for Education and Psychology. Vol. 23: Iss. 2, Article \n.1'

In [9]:
from utils.various import arabic_print
sample = data.sample(1).iloc[0]
abstarct,kws,page1,page2 = sample["abstract_ar"] ,sample["keywords"],sample["page1"],sample["page2"]
arabic_print("abstarct",abstarct,"kws",kws,"pages",page1,page2)


### Clean and Fill Data
Since this is a heuristic approach, some abstracts might be missing. We will fill missing abstracts with a placeholder to ensure the dataset structure remains valid for subsequent processing steps.

In [10]:
print(data["abstract_ar"].isnull().sum())
print(data["keywords"].isnull().sum())

60
130


In [11]:
# Drop rows where "abstract_ar" or "keywords" are NaN
data = data.dropna(subset=["abstract_ar", "keywords"])

# Verify the changes
print(f"Nulls in abstract_ar: {data['abstract_ar'].isnull().sum()}")
print(f"Nulls in keywords: {data['keywords'].isnull().sum()}")

Nulls in abstract_ar: 0
Nulls in keywords: 0


In [16]:
# List of columns to keep
columns_to_keep = [
    'article_id', 'authors', 'authors_en', 
    'abstract_ar', 'source', 'path', 'keywords'
]

# Overwrite the dataframe with only these columns
data = data[columns_to_keep]

In [17]:
data.to_csv("05-heuristic_extraction.csv", index=False)